In [1]:
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

In [2]:
num_classes = 4
image_size = 224
batch_size_training = 10
batch_size_validation = 1

In [3]:
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input,)

#data_generator = ImageDataGenerator(preprocessing_function=preprocess_input,)

In [4]:
train_generator = data_generator.flow_from_directory(
    'New folder/train',
    target_size = (image_size, image_size),
    batch_size = batch_size_training,
    class_mode = 'categorical'
    )

validation_generator = data_generator.flow_from_directory(
    'New folder/validation',
    target_size = (image_size, image_size),
    batch_size = batch_size_training,
    class_mode = 'categorical'
    )

Found 304 images belonging to 4 classes.
Found 9 images belonging to 4 classes.


In [5]:
#adding VGG16 model
from xml.etree.ElementInclude import include

base_model = VGG16(weights="imagenet", include_top=False, input_shape=[image_size, image_size, 3], pooling='max')
base_model.trainable = False

In [6]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [7]:

from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(4096, activation='relu')
dense_layer_2 = layers.Dense(4096, activation='relu')
prediction_layer = layers.Dense(4, activation='softmax')


model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 4096)              2101248   
                                                                 
 dense_1 (Dense)             (None, 4096)              16781312  
                                                                 
 dense_2 (Dense)             (None, 4)                 16388     
                                                                 
Total params: 33,613,636
Trainable params: 18,898,948
Non-trainable params: 14,714,688
_________________________________________________________________


In [9]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

steps_per_epoch_train = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
number_epochs = 2




#model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])

In [10]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

#early stopping is not used
fit_history = model.fit(train_generator, epochs=number_epochs,verbose=1, validation_data=validation_generator, callbacks=[es])

Epoch 1/2
31/31 [==============================] - 32s 1s/step - loss: 28.4629 - accuracy: 0.8421 - val_loss: 1.7438 - val_accuracy: 0.8889
Epoch 2/2
31/31 [==============================] - 34s 1s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 70.8425 - val_accuracy: 0.6667


In [11]:
model.save('model/VGG16_latter_training/final_with_one_image_and3dense4096.h5')

## RUN THIS MODEL

In [2]:
from keras.models import load_model
vgg16_saved = load_model('model/VGG16_latter_training/final_with_one_image_and3dense4096.h5')

In [3]:
from keras.models import Model
basemodel1 = Model(inputs=vgg16_saved.input, outputs=vgg16_saved.get_layer('dense').output)

In [4]:
import numpy as np
from numpy.linalg import norm

In [5]:
#predicting
from tensorflow.keras.preprocessing.image import load_img
# load an image from file
image1 = load_img('New folder/test/test4.jpg', target_size=(224, 224))
image2 = load_img('New folder/test/test3.png', target_size=(224, 224))


from tensorflow.keras.preprocessing.image import img_to_array
image1 = img_to_array(image1)
image2 = img_to_array(image2)

# reshape data for the model
image1 = image1.reshape((1, image1.shape[0], image1.shape[1], image1.shape[2]))
image2 = image2.reshape((1, image2.shape[0], image2.shape[1], image2.shape[2]))


from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image1 = preprocess_input(image1)
image2 = preprocess_input(image2)

# predict the probability across all output classes
yhat1_base = basemodel1.predict(image1)
yhat2_base = basemodel1.predict(image2)

#cosine similarity
A = yhat1_base[0]
B = yhat2_base[0]

cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)

#0.78 on model 3


1/1 [==============================] - 0s 206ms/step
Cosine Similarity: 0.820057


In [6]:
#predicting
from tensorflow.keras.preprocessing.image import load_img
# load an image from file
image1 = load_img('New folder/test/test4.jpg', target_size=(224, 224))
image2 = load_img('New folder/test/test.png', target_size=(224, 224))


from tensorflow.keras.preprocessing.image import img_to_array
image1 = img_to_array(image1)
image2 = img_to_array(image2)

# reshape data for the model
image1 = image1.reshape((1, image1.shape[0], image1.shape[1], image1.shape[2]))
image2 = image2.reshape((1, image2.shape[0], image2.shape[1], image2.shape[2]))


from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image1 = preprocess_input(image1)
image2 = preprocess_input(image2)

# predict the probability across all output classes
yhat1_base = basemodel1.predict(image1)
yhat2_base = basemodel1.predict(image2)

#cosine similarity
A = yhat1_base[0]
B = yhat2_base[0]

cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)

1/1 [==============================] - 0s 145ms/step
Cosine Similarity: 0.59367347


In [7]:
#predicting
from tensorflow.keras.preprocessing.image import load_img
# load an image from file
image1 = load_img('New folder/test/test.png', target_size=(224, 224))
image2 = load_img('New folder/test/test3.png', target_size=(224, 224))


from tensorflow.keras.preprocessing.image import img_to_array
image1 = img_to_array(image1)
image2 = img_to_array(image2)

# reshape data for the model
image1 = image1.reshape((1, image1.shape[0], image1.shape[1], image1.shape[2]))
image2 = image2.reshape((1, image2.shape[0], image2.shape[1], image2.shape[2]))


from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image1 = preprocess_input(image1)
image2 = preprocess_input(image2)

# predict the probability across all output classes
yhat1_base = vgg16_saved.predict(image1)
yhat2_base = vgg16_saved.predict(image2)

#cosine similarity
A = yhat1_base[0]
B = yhat2_base[0]

cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)


1/1 [==============================] - 0s 206ms/step
Cosine Similarity: 1.0


In [8]:
#predicting
from tensorflow.keras.preprocessing.image import load_img
# load an image from file
image1 = load_img('New folder/test/test4.jpg', target_size=(224, 224))
image2 = load_img('New folder/test/test3.png', target_size=(224, 224))


from tensorflow.keras.preprocessing.image import img_to_array
image1 = img_to_array(image1)
image2 = img_to_array(image2)

# reshape data for the model
image1 = image1.reshape((1, image1.shape[0], image1.shape[1], image1.shape[2]))
image2 = image2.reshape((1, image2.shape[0], image2.shape[1], image2.shape[2]))


from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image1 = preprocess_input(image1)
image2 = preprocess_input(image2)

# predict the probability across all output classes
yhat1_base = basemodel1.predict(image1)
yhat2_base = basemodel1.predict(image2)

#cosine similarity
A = yhat1_base[0]
B = yhat2_base[0]

cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)

1/1 [==============================] - 0s 151ms/step
Cosine Similarity: 0.820057


In [9]:
#predicting
from tensorflow.keras.preprocessing.image import load_img
# load an image from file
image1 = load_img('New folder/test/test3.png', target_size=(224, 224))
image2 = load_img('New folder/test/test2.jpg', target_size=(224, 224))


from tensorflow.keras.preprocessing.image import img_to_array
image1 = img_to_array(image1)
image2 = img_to_array(image2)

# reshape data for the model
image1 = image1.reshape((1, image1.shape[0], image1.shape[1], image1.shape[2]))
image2 = image2.reshape((1, image2.shape[0], image2.shape[1], image2.shape[2]))


from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image1 = preprocess_input(image1)
image2 = preprocess_input(image2)

# predict the probability across all output classes
yhat1_base = basemodel1.predict(image1)
yhat2_base = basemodel1.predict(image2)

#cosine similarity
A = yhat1_base[0]
B = yhat2_base[0]

cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)

1/1 [==============================] - 0s 150ms/step
Cosine Similarity: 0.45856088
